# Домашнее задание
Использование Facial Landmarks в качестве Feature Estimator

Цель:
Создадить свой пайплайн Facial Recognition.

Описание/Пошаговая инструкция выполнения домашнего задания:
- Выберите Facial Detector по вкусу.
- Выполните Face Alignment.
- На это натравите Facial Landmarks Detector по выбору.
- На этом обучите классификатор на предпочитаемом датасете.

 # Пайплайн:
 ## __Facial Detector → Face Alignment → Facial Landmarks → Классификатор__

### 1. Выбор Facial Detector технологии
MediaPipe Face Detection - актуальный, бесплатный, функциональный, не требовательный

In [1]:
import mediapipe

### 2. Выбор Face Alignment решения
MediaPipe Face Mesh - 468 точек. Более чем достаточно для любых задач

In [2]:
face_detection = mediapipe.solutions.face_mesh # солюшн по лицу
drawing = mediapipe.solutions.drawing_utils
show_mask = False

face_mesh = face_detection.FaceMesh(
    static_image_mode=False,
    max_num_faces=1,
    refine_landmarks=True,
    min_detection_confidence=0.5)

### 3. Наложение маски ~ извлечение маски


In [9]:
import cv2
import numpy as np

# Извлечение маски
def get_face_landmarks(image):
    image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    results = face_mesh.process(image_rgb)
    if not results.multi_face_landmarks:
        return None, None  
    face_landmarks = results.multi_face_landmarks[0]
    landmarks_array = np.array([[lm.x, lm.y, lm.z] for lm in face_landmarks.landmark]).flatten()
    return face_landmarks, landmarks_array

In [4]:
import os
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import joblib

X = []
y = []
label_to_name = {}

dataset_path = "Homework/webcam_captures/"

for idx, person_name in enumerate(os.listdir(dataset_path)):
    label_to_name[idx] = person_name
    person_dir = os.path.join(dataset_path, person_name)
    for img_file in os.listdir(person_dir):
        img_path = os.path.join(person_dir, img_file)
        img = cv2.imread(img_path)
        if img is None:
            continue
        landmarks = get_face_landmarks(img)
        if landmarks is not None:
            X.append(landmarks)
            y.append(idx)

X = np.array(X)
y = np.array(y)

# Разделение трейн/тест
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


### 4. Обучение классификатора

In [5]:
# Обучение классификатора
clf = SVC(kernel='linear', probability=True)
clf.fit(X_train, y_train)

# Сохранение модели
joblib.dump(clf, 'face_classifier.pkl')
joblib.dump(label_to_name, 'label_to_name.pkl')


['label_to_name.pkl']

In [ ]:
# Загрузка модели
clf = joblib.load('face_classifier.pkl')
label_to_name = joblib.load('label_to_name.pkl')

### 5. Инференс

In [ ]:
cap = cv2.VideoCapture(0)

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break
    #frame = cv2.resize(frame, (640, 480))

    face_landmarks, landmarks_array = get_face_landmarks(frame)

    if face_landmarks is not None:
        # Отрисовка только точек лица
        if show_mask:
            drawing.draw_landmarks(
                image=frame,
                landmark_list=face_landmarks,
                connections=None,
                landmark_drawing_spec=drawing.DrawingSpec(color=(0, 255, 0), thickness=2, circle_radius=1),
                connection_drawing_spec=None
            )

        # Предсказание класса
        prediction = clf.predict([landmarks_array])
        name = label_to_name[prediction[0]]
        proba = clf.predict_proba([landmarks_array]).max()

        # Отображение имени
        text = f"{name} ({proba*100:.1f}%)"
        cv2.putText(frame, text, (10, 50),
                    cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)

        # Cсостояния маски
        mask_status = "Mask: ON" if show_mask else "Mask: OFF"
        cv2.putText(frame, mask_status, (10, 90),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 0, 255), 2)

    cv2.imshow("NAOMIvsME", frame)

    key = cv2.waitKey(1) & 0xFF
    if key == ord('q'):
        break
    elif key == ord('r'):
        show_mask = not show_mask
cap.release()
cv2.destroyAllWindows()